# Solution

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,Date,Name,Value
0,2022-11-24 22:34:55,WZMHH,NaN
1,2022-05-30 14:44:47,DIXFU,NaN
2,2022-10-22 08:38:49,OHCUI,0.637045
3,2022-12-12 17:40:29,GEVAZ,0.529207
4,2022-06-13 08:00:40,GBRDD,0.376432


## Pandas `.to_sql()`

At first the [`.to_sql()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html) function seems promising

In [2]:
from sqlalchemy import create_engine, text
engine = create_engine('sqlite://', echo=False)

In [3]:
df.to_sql('test', engine, index=False)

100

In [4]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM test;"))
result.fetchall()[:5]

[('2022-11-24 22:34:55', 'WZMHH', None),
 ('2022-05-30 14:44:47', 'DIXFU', None),
 ('2022-10-22 08:38:49', 'OHCUI', 0.6370445113549524),
 ('2022-12-12 17:40:29', 'GEVAZ', 0.5292072971005523),
 ('2022-06-13 08:00:40', 'GBRDD', 0.376431751315491)]

Almost, but not quite. How to intercept the transaction before it is executed and inserted into the DB?

## String Mapping

Instead, I created a function function to maps the `str` method to each row. 

In [5]:
import re
def df_to_sql_bulk_insert(input_df, table, date_columns=None):
    """https://stackoverflow.com/questions/31071952/generate-sql-statements-from-a-pandas-dataframe/70585493#70585493"""
    df = input_df.copy()
    if date_columns:
        for name in date_columns:
            df[name] = df[name].copy().apply(str)
    columns = ", ".join(df.columns)
    tuples = map(str, df.itertuples(index=False, name=None))
    values = re.sub(r"(?<=\W)(nan|None)(?=\W)", "NULL", (",\n" + " ").join(tuples))
    values = values.replace("'NULL'", "NULL")
    return f"INSERT INTO {table} ({columns})\nVALUES\n {values};\n"

In [6]:
t = df_to_sql_bulk_insert(df, 'SAMPLE_TABLE', date_columns=['Date'])

In [7]:
print(t[:518])

INSERT INTO SAMPLE_TABLE (Date, Name, Value)
VALUES
 ('2022-11-24 22:34:55', 'WZMHH', NULL),
 ('2022-05-30 14:44:47', 'DIXFU', NULL),
 ('2022-10-22 08:38:49', 'OHCUI', 0.6370445113549524),
 ('2022-12-12 17:40:29', 'GEVAZ', 0.5292072971005523),
 ('2022-06-13 08:00:40', 'GBRDD', 0.376431751315491),
 ('2022-01-16 17:20:30', 'BCSVR', 0.0084276081414927),
 ('2022-04-11 13:16:44', 'VNNGF', 0.9619565585865116),
 ('2022-07-18 13:25:32', 'RVKHM', 0.901262330417864),
 ('2022-07-08 17:10:27', 'QSZJI', 0.4412973836361636),
 


## Conclusion

I have solved this problem using the function defined above, **BUT** I don't like it. I want to leverage pandas to get the same results. 

I have done some investigation into the [pandas/io/sql.py](https://github.com/pandas-dev/pandas/blob/main/pandas/io/sql.py) code but I haven't figured it out yet. I think [sqlalchemy](https://www.sqlalchemy.org/) would be helpful, but I am not so familiar with the library. 

Any advice or help is appreciated!

Create a new notebook with your solution and submit a PR.